# Phase 6: Pyramiding & Scaling

Techniques used by the most successful traders for capital growth.

This notebook covers:
- **Risk-managed pyramiding** — adding to winners safely
- **Scaling in/out** — gradual position building and profit-taking
- **Trend following with diversification** — multi-asset portfolio construction
- **Dynamic position management** — conviction-based sizing

---

```bash
pip install pandas numpy matplotlib
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

def generate_ohlcv(n=500, mu=0.12, sigma=0.22, start=100.0):
    dt = 1/252
    returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), n)
    close = start * np.exp(np.cumsum(returns))
    noise = sigma * np.sqrt(dt) * 0.5
    dates = pd.date_range('2023-01-01', periods=n, freq='B')
    return pd.DataFrame({
        'open': close * (1 + np.random.normal(0, noise, n)),
        'high': close * (1 + np.abs(np.random.normal(0, noise, n))),
        'low':  close * (1 - np.abs(np.random.normal(0, noise, n))),
        'close': close,
        'volume': np.random.lognormal(10, 1, n).astype(int)
    }, index=dates)

def ema(s, p): return s.ewm(span=p, adjust=False).mean()
def atr(df, p=14):
    tr = pd.concat([df['high']-df['low'], abs(df['high']-df['close'].shift()), abs(df['low']-df['close'].shift())], axis=1).max(axis=1)
    return tr.rolling(p).mean()

df = generate_ohlcv(500)
print(f"Generated {len(df)} candles")

---
## 6.1 Pyramiding: Adding to Winners

Pyramiding means adding to a winning position as the trend continues. The key rule: **each add-on must risk less than the original entry** so that total risk stays bounded.

### Pyramiding Rules:
1. Only add to winning positions
2. Each add-on is smaller than the previous
3. Move stop-loss to protect accumulated profits
4. Total risk per position never exceeds a maximum (e.g., 2% of equity)

In [ ]:
@dataclass
class PyramidLevel:
    entry_price: float
    quantity: float
    stop_price: float


class PyramidingStrategy:
    def __init__(
        self,
        equity: float = 100_000,
        risk_per_level: float = 0.005,   # 0.5% risk per pyramid level
        max_levels: int = 4,              # max 4 add-ons
        atr_stop_mult: float = 2.0,      # ATR multiplier for stops
        add_threshold_atr: float = 1.0,  # add when price moves 1 ATR in favor
        leverage: float = 3.0
    ):
        self.equity = equity
        self.risk_per_level = risk_per_level
        self.max_levels = max_levels
        self.atr_stop_mult = atr_stop_mult
        self.add_threshold_atr = add_threshold_atr
        self.leverage = leverage
        self.levels: list[PyramidLevel] = []
        self.side = 0  # 1=long, -1=short

    @property
    def total_quantity(self):
        return sum(l.quantity for l in self.levels)

    @property
    def avg_entry(self):
        if not self.levels:
            return 0
        total_cost = sum(l.entry_price * l.quantity for l in self.levels)
        return total_cost / self.total_quantity

    @property
    def total_risk_pct(self):
        return self.risk_per_level * len(self.levels)

    def should_enter(self, fast_ma, slow_ma) -> int:
        if fast_ma > slow_ma:
            return 1
        elif fast_ma < slow_ma:
            return -1
        return 0

    def should_add(self, price, atr_val) -> bool:
        if len(self.levels) >= self.max_levels:
            return False
        if not self.levels:
            return False
        last_entry = self.levels[-1].entry_price
        if self.side == 1 and price > last_entry + self.add_threshold_atr * atr_val:
            return True
        if self.side == -1 and price < last_entry - self.add_threshold_atr * atr_val:
            return True
        return False

    def add_level(self, price, atr_val):
        risk_amount = self.equity * self.risk_per_level
        stop_distance = atr_val * self.atr_stop_mult
        quantity = (risk_amount / stop_distance) * self.leverage

        # Decreasing size: each level is smaller
        level_num = len(self.levels)
        scale = 1.0 / (1 + level_num * 0.25)  # 100%, 80%, 67%, 57%
        quantity *= scale

        stop = price - self.side * stop_distance
        self.levels.append(PyramidLevel(price, quantity, stop))

        # Move all stops to the tightest level
        if self.side == 1:
            best_stop = max(l.stop_price for l in self.levels)
            for l in self.levels:
                l.stop_price = best_stop
        else:
            best_stop = min(l.stop_price for l in self.levels)
            for l in self.levels:
                l.stop_price = best_stop

    def check_stop(self, price) -> bool:
        if not self.levels:
            return False
        if self.side == 1 and price <= self.levels[0].stop_price:
            return True
        if self.side == -1 and price >= self.levels[0].stop_price:
            return True
        return False

    def close_all(self, price) -> float:
        pnl = sum(self.side * l.quantity * (price - l.entry_price) for l in self.levels)
        self.levels = []
        self.side = 0
        return pnl


def run_pyramid_backtest(df, equity=100_000, max_levels=4, leverage=3.0):
    strat = PyramidingStrategy(equity=equity, max_levels=max_levels, leverage=leverage)
    df = df.copy()
    df['fast_ma'] = ema(df['close'], 20)
    df['slow_ma'] = ema(df['close'], 50)
    df['atr'] = atr(df)

    equity_history = [equity]
    levels_history = [0]
    trades = []

    for i in range(50, len(df)):
        price = df['close'].iloc[i]
        atr_val = df['atr'].iloc[i]
        if np.isnan(atr_val):
            equity_history.append(strat.equity)
            levels_history.append(0)
            continue

        # Check stop
        if strat.check_stop(price):
            pnl = strat.close_all(price)
            strat.equity += pnl
            trades.append({'type': 'stop', 'pnl': pnl, 'levels': len(strat.levels)})

        # Check for new entry
        if strat.side == 0:
            signal = strat.should_enter(df['fast_ma'].iloc[i], df['slow_ma'].iloc[i])
            if signal != 0:
                strat.side = signal
                strat.add_level(price, atr_val)

        # Check for pyramid add-on
        elif strat.should_add(price, atr_val):
            strat.add_level(price, atr_val)

        # Check for trend reversal
        signal = strat.should_enter(df['fast_ma'].iloc[i], df['slow_ma'].iloc[i])
        if strat.side != 0 and signal != strat.side:
            pnl = strat.close_all(price)
            strat.equity += pnl
            trades.append({'type': 'reversal', 'pnl': pnl})

        # Track equity (including unrealized)
        unrealized = sum(strat.side * l.quantity * (price - l.entry_price) for l in strat.levels)
        equity_history.append(strat.equity + unrealized)
        levels_history.append(len(strat.levels))

    return pd.Series(equity_history, index=df.index[:len(equity_history)]), \
           pd.Series(levels_history, index=df.index[:len(levels_history)]), \
           pd.DataFrame(trades)


# Compare pyramid vs no pyramid
eq_pyramid, levels, pyramid_trades = run_pyramid_backtest(df, max_levels=4, leverage=3)
eq_single, _, single_trades = run_pyramid_backtest(df, max_levels=1, leverage=3)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

ax1.plot(eq_pyramid.index, eq_pyramid, color='green', linewidth=1.5, label='With Pyramiding (4 levels)')
ax1.plot(eq_single.index, eq_single, color='blue', linewidth=1.5, label='Single Entry')
ax1.axhline(y=100_000, color='gray', linestyle='--', alpha=0.5)
ax1.set_ylabel('Equity ($)')
ax1.set_title('Pyramiding vs Single Entry (3x Leverage)', fontsize=14)
ax1.legend()

ax2.fill_between(levels.index, levels, 0, color='orange', alpha=0.4, step='post')
ax2.set_ylabel('Pyramid Levels')
ax2.set_xlabel('Date')
ax2.set_ylim(0, 5)

plt.tight_layout()
plt.show()

print(f"Pyramiding: ${eq_pyramid.iloc[-1]:,.2f} ({(eq_pyramid.iloc[-1]/100_000-1):.1%})")
print(f"Single entry: ${eq_single.iloc[-1]:,.2f} ({(eq_single.iloc[-1]/100_000-1):.1%})")

### Exercise 6.1

1. Change `max_levels` to 2 and 6. How does the number of add-ons affect risk and return?
2. Change `risk_per_level` from 0.5% to 1%. What happens to drawdowns?
3. Implement "anti-pyramiding": reduce position size as the trade goes further in your favor (scale out).

In [ ]:
# YOUR CODE HERE


---
## 6.2 Scale In / Scale Out

Instead of entering/exiting all at once, build the position gradually and take profits at predetermined levels.

In [ ]:
def scale_in_out_simulation(
    prices: pd.Series,
    entry_idx: int,
    direction: int = 1,
    # Scale-in: split entry into 3 tranches
    entry_splits: list = [0.50, 0.30, 0.20],  # 50%, 30%, 20%
    entry_spacing_pct: float = 0.01,  # add 1% apart
    # Scale-out: take profits at levels
    exit_levels: list = [0.03, 0.05, 0.08],  # 3%, 5%, 8%
    exit_splits: list = [0.33, 0.33, 0.34],
    stop_loss_pct: float = 0.02,
    total_position: float = 10_000,
    leverage: float = 5.0
) -> dict:
    entry_price = prices.iloc[entry_idx]
    entries = []
    exits = []
    position_value = 0
    avg_entry = 0
    entry_tranches_filled = 0
    exit_tranches_filled = 0
    equity = total_position
    equity_path = []

    # Calculate entry targets
    entry_targets = [
        entry_price * (1 - direction * i * entry_spacing_pct)
        for i in range(len(entry_splits))
    ]

    for i in range(entry_idx, min(entry_idx + 100, len(prices))):
        price = prices.iloc[i]

        # Scale in
        if entry_tranches_filled < len(entry_splits):
            target = entry_targets[entry_tranches_filled]
            if (direction == 1 and price <= target) or (direction == -1 and price >= target):
                tranche_value = total_position * entry_splits[entry_tranches_filled] * leverage
                tranche_qty = tranche_value / price
                entries.append({'price': price, 'value': tranche_value, 'idx': i})
                old_value = position_value
                position_value += tranche_value
                avg_entry = (avg_entry * old_value + price * tranche_value) / position_value if position_value > 0 else price
                entry_tranches_filled += 1

        # Track equity
        if position_value > 0:
            pnl = direction * position_value * (price - avg_entry) / avg_entry
            equity_path.append(total_position + pnl)
        else:
            equity_path.append(total_position)

        # Scale out
        if position_value > 0 and exit_tranches_filled < len(exit_levels):
            target_pct = exit_levels[exit_tranches_filled]
            gain = direction * (price - avg_entry) / avg_entry
            if gain >= target_pct:
                exit_value = position_value * exit_splits[exit_tranches_filled]
                exits.append({'price': price, 'value': exit_value, 'gain': gain, 'idx': i})
                position_value -= exit_value
                exit_tranches_filled += 1

        # Stop loss on remaining position
        if position_value > 0:
            loss = direction * (price - avg_entry) / avg_entry
            if loss <= -stop_loss_pct:
                exits.append({'price': price, 'value': position_value, 'gain': loss, 'idx': i})
                position_value = 0
                break

    return {
        'entries': entries, 'exits': exits,
        'avg_entry': avg_entry, 'equity_path': equity_path,
        'remaining_position': position_value
    }


result = scale_in_out_simulation(df['close'], entry_idx=100, direction=1)

fig, ax = plt.subplots(figsize=(14, 6))
window = df['close'].iloc[100:200]
ax.plot(range(len(window)), window.values, color='steelblue', linewidth=1)

for e in result['entries']:
    idx = e['idx'] - 100
    ax.scatter(idx, e['price'], color='green', s=100, zorder=5, marker='^')
    ax.annotate(f"Buy ${e['value']:,.0f}", (idx, e['price']), fontsize=8, color='green')

for e in result['exits']:
    idx = e['idx'] - 100
    ax.scatter(idx, e['price'], color='red', s=100, zorder=5, marker='v')
    ax.annotate(f"Sell ({e['gain']:.1%})", (idx, e['price']), fontsize=8, color='red')

ax.axhline(y=result['avg_entry'], color='orange', linestyle='--', alpha=0.5, label=f"Avg Entry ${result['avg_entry']:.2f}")
ax.set_title('Scale In / Scale Out Example', fontsize=14)
ax.set_ylabel('Price ($)')
ax.legend()
plt.tight_layout()
plt.show()

print(f"Entries: {len(result['entries'])}, Exits: {len(result['exits'])}")
print(f"Avg entry price: ${result['avg_entry']:.2f}")

---
## 6.3 Diversified Trend Following Portfolio

Run trend following across multiple uncorrelated assets to smooth out returns.

In [ ]:
# Generate diverse asset universe
assets = {
    'US_Equity': generate_ohlcv(500, mu=0.10, sigma=0.18),
    'EU_Equity': generate_ohlcv(500, mu=0.07, sigma=0.20),
    'Gold':      generate_ohlcv(500, mu=0.05, sigma=0.15),
    'Oil':       generate_ohlcv(500, mu=0.03, sigma=0.35),
    'BTC':       generate_ohlcv(500, mu=0.20, sigma=0.60),
    'Bonds':     generate_ohlcv(500, mu=0.03, sigma=0.08),
    'EM_Equity': generate_ohlcv(500, mu=0.08, sigma=0.25),
    'JPY':       generate_ohlcv(500, mu=0.01, sigma=0.10),
}

# Simple trend following: long if above 100-day MA, short if below
def trend_signal(prices, lookback=100):
    ma = prices.rolling(lookback).mean()
    signal = pd.Series(0, index=prices.index)
    signal[prices > ma] = 1
    signal[prices < ma] = -1
    return signal

# Equal risk contribution (volatility-weighted)
target_risk = 0.10  # 10% target portfolio volatility
leverage_cap = 3.0

portfolio_returns = pd.DataFrame()
individual_returns = pd.DataFrame()

for name, data in assets.items():
    returns = data['close'].pct_change()
    vol = returns.rolling(60).std() * np.sqrt(252)
    signal = trend_signal(data['close'])

    # Volatility-targeted position sizing
    weight = (target_risk / len(assets)) / vol
    weight = weight.clip(upper=leverage_cap / len(assets))

    strat_return = signal.shift(1) * returns * weight
    individual_returns[name] = strat_return

portfolio_returns = individual_returns.sum(axis=1).dropna()

fig, axes = plt.subplots(2, 1, figsize=(14, 9))

# Individual and portfolio equity curves
for name in individual_returns.columns:
    cum = (1 + individual_returns[name].dropna()).cumprod()
    axes[0].plot(cum.index, cum, alpha=0.4, linewidth=0.8, label=name)

cum_port = (1 + portfolio_returns).cumprod()
axes[0].plot(cum_port.index, cum_port, color='black', linewidth=2.5, label='PORTFOLIO')
axes[0].axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_ylabel('Cumulative Return')
axes[0].set_title('Diversified Trend Following Portfolio', fontsize=14)
axes[0].legend(fontsize=8, ncol=3)

# Rolling Sharpe
rolling_sharpe = portfolio_returns.rolling(126).mean() / portfolio_returns.rolling(126).std() * np.sqrt(252)
axes[1].plot(rolling_sharpe.index, rolling_sharpe, color='steelblue')
axes[1].axhline(y=0, color='red', linewidth=0.5)
axes[1].axhline(y=1, color='green', linestyle='--', alpha=0.5)
axes[1].set_ylabel('Rolling 6m Sharpe')
axes[1].set_xlabel('Date')

plt.tight_layout()
plt.show()

ann_ret = portfolio_returns.mean() * 252
ann_vol = portfolio_returns.std() * np.sqrt(252)
sharpe = ann_ret / ann_vol
print(f"Portfolio annual return: {ann_ret:.2%}")
print(f"Portfolio annual volatility: {ann_vol:.2%}")
print(f"Portfolio Sharpe ratio: {sharpe:.2f}")

# Correlation matrix
print("\nAsset return correlations:")
corr = individual_returns.corr().round(2)
print(corr)

### Exercise 6.3

1. Remove BTC from the portfolio. How does it affect the Sharpe ratio? BTC adds returns but also risk — is the trade-off worth it?
2. Change the trend lookback from 100 to 200 days. Longer-term trend following — what changes?
3. Implement a correlation filter: reduce weight for assets that become highly correlated (>0.7) during recent periods.

In [ ]:
# YOUR CODE HERE


---
## 6.4 Comprehension Check

1. You pyramid 4 times at 0.5% risk each. What's your total position risk? Why is the actual risk often less than 2%?
2. Why do successful traders scale OUT (take partial profits) instead of exiting all at once?
3. A portfolio of 8 uncorrelated assets each with a Sharpe of 0.5 has a portfolio Sharpe of approximately 0.5 × √8 ≈ 1.4. Verify this with the simulation above.
4. When does pyramiding hurt performance? (Hint: think about range-bound markets)
5. Why is volatility-based position sizing better than equal-dollar sizing across a diverse portfolio?

In [ ]:
# YOUR ANSWERS HERE
